In [1]:
!pip install ase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 5.3 MB/s 


In [ ]:
from ase.db import connect
db = connect('abc.db')

In [ ]:
from ase.io import read
b = read('abc.db@v3,abc=H')

In [ ]:
from ase.ga.utilities import get_all_atom_types
from ase.constraints import FixAtoms
import numpy as np
from ase.build import fcc111

db_file = 'gadb.db'

# create the surface
slab = fcc111('Au', size=(4, 4, 1), vacuum=10.0, orthogonal=True)
slab.set_constraint(FixAtoms(mask=len(slab) * [True]))

# define the volume in which the adsorbed cluster is optimized
# the volume is defined by a corner position (p0)
# and three spanning vectors (v1, v2, v3)
pos = slab.get_positions()
cell = slab.get_cell()

In [ ]:
pos

array([[ 0.        ,  0.        , 10.        ],
       [ 2.88499567,  0.        , 10.        ],
       [ 5.76999133,  0.        , 10.        ],
       [ 8.654987  ,  0.        , 10.        ],
       [ 1.44249783,  2.49847954, 10.        ],
       [ 4.3274935 ,  2.49847954, 10.        ],
       [ 7.21248917,  2.49847954, 10.        ],
       [10.09748484,  2.49847954, 10.        ],
       [ 0.        ,  4.99695908, 10.        ],
       [ 2.88499567,  4.99695908, 10.        ],
       [ 5.76999133,  4.99695908, 10.        ],
       [ 8.654987  ,  4.99695908, 10.        ],
       [ 1.44249783,  7.49543861, 10.        ],
       [ 4.3274935 ,  7.49543861, 10.        ],
       [ 7.21248917,  7.49543861, 10.        ],
       [10.09748484,  7.49543861, 10.        ]])

In [ ]:
cell

Cell([11.539982668964457, 9.993918150555366, 20.0])

In [ ]:
p0 = np.array([0., 0., max(pos[:, 2]) + 2.])
v1 = cell[0, :] * 0.8
v2 = cell[1, :] * 0.8
v3 = cell[2, :]
v3[2] = 3.

# Define the composition of the atoms to optimize
atom_numbers = 2 * [47] + 2 * [79]

# define the closest distance two atoms of a given species can be to each other
unique_atom_types = get_all_atom_types(slab, atom_numbers)
blmin = closest_distances_generator(atom_numbers=unique_atom_types,
                                    ratio_of_covalent_radii=0.7)